# Part 1

In [ ]:
boss_hp = 71
boss_dmg = 10

class GameSimulation(object):
    
    def __init__(self, boss_hp, boss_dmg):
        self.boss_hp = boss_hp
        self.boss_dmg = boss_dmg
        self.player_hp = player_hp

# Part 2

9765625